### Parameters to update
##### Make sure notebook is attached to a lakehouse to log results
##### Two tables will be created. QueryResults and RunResults

In [1]:
FabricDWWorkspaceName = 'WS_DW_Query_Cost_Test_Take_Nine'
FabricDWName = 'WH_SampleData'
ConcurrencyNum = 1 # This should be equal or greater than the length of the dataframe with the queryies defined below
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 3, Finished, Available, Finished)

In [2]:
print(f"{FabricDWWorkspaceName=}")
print(f"{FabricDWName=}")
print(f"{ConcurrencyNum=}")
print(f"{CapacityMetricsWorkspace=}")
print(f"{CapacityMetricsDataset=}")

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 4, Finished, Available, Finished)

FabricDWWorkspaceName='WS_DW_Query_Cost_Test_Take_Nine'
FabricDWName='WH_SampleData'
ConcurrencyNum=1
CapacityMetricsWorkspace='Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset='Fabric Capacity Metrics'


In [3]:
queryList = [
    # Transaction
    'SELECT COUNT(*) FROM FactTransaction'
    ,'''SELECT	COUNT(*)
FROM	FactTransaction AS ft
JOIN	DimDate AS d
ON		d.Date = ft.DateKey
JOIN	DimPaymentMethod AS pm
ON		pm.PaymentMethodKey= ft.PaymentMethodKey
JOIN	DimTransactionType AS tt
ON		tt.TransactionTypeKey = ft.TransactionTypeKey
JOIN	DimSupplier AS s
ON		s.SupplierKey = ft.SupplierKey
JOIN	DimCustomer AS cu
ON		cu.CustomerKey = ft.CustomerKey
JOIN	DimCustomer AS cuBill
ON		cuBill.CustomerKey = ft.BillToCustomerKey'''
    # Order
    ,'SELECT COUNT(*) FROM FactOrder'
    ,'''SELECT	COUNT(*)
FROM	FactOrder AS fo
JOIN	DimDate AS dOrder
ON		dOrder.Date = fo.OrderDateKey
LEFT JOIN	DimDate AS dPicked
ON		dPicked.Date = fo.PickedDateKey
JOIN	DimStockItem AS si
ON		si.StockItemKey = fo.StockItemKey
JOIN	DimCity AS c
ON		c.CityKey = fo.CityKey
JOIN	DimEmployee AS e
ON		e.EmployeeKey = fo.SalespersonKey
JOIN	DimEmployee AS ePicker
ON		ePicker.EmployeeKey = fo.PickerKey
JOIN	DimCustomer AS cu
ON		cu.CustomerKey = fo.CustomerKey'''
    # FactMovement
    ,'SELECT COUNT(*) FROM FactMovement'
    ,'''SELECT	COUNT(*)
FROM	FactMovement AS fm
JOIN	DimDate AS d
ON		d.Date = fm.DateKey
JOIN	DimStockItem AS si
ON		si.StockItemKey = fm.StockItemKey
JOIN	DimTransactionType AS tt
ON		tt.TransactionTypeKey = fm.TransactionTypeKey
JOIN	DimSupplier AS s
ON		s.SupplierKey = fm.SupplierKey
JOIN	DimCustomer AS c
ON		c.CustomerKey = fm.CustomerKey'''
    # FactPurchase
    ,'SELECT COUNT(*) FROM FactPurchase'
    ,'''SELECT	COUNT(*)
FROM	FactPurchase AS fp
JOIN	DimDate AS d
ON		d.Date = fp.DateKey
JOIN	DimStockItem AS si
ON		si.StockItemKey = fp.StockItemKey
JOIN	DimSupplier AS s
ON		s.SupplierKey = fp.SupplierKey'''
    # StockHolding
    ,'SELECT COUNT(*) FROM FactStockHolding'
    ,'''SELECT	COUNT(*)
FROM	FactStockHolding AS fsh
JOIN	DimStockItem AS si
ON		si.StockItemKey = fsh.StockItemKey'''
    # Stored Procedures
    ,'''EXEC sp_Ingest'''
    ,'''EXEC sp_Query'''
    # Query with multiple statements
    ,'''IF OBJECT_ID('dbo.DimDate', 'U') IS NOT NULL DROP TABLE dbo.DimDate; CREATE TABLE dbo.DimDate AS SELECT * FROM LH_SampleData.dbo.DimDate'''
]

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 5, Finished, Available, Finished)

In [4]:
executorCoreCnt = int(spark.conf.get('spark.executor.cores'))
executorInstances = int(spark.conf.get('spark.executor.instances'))
maxConcurrency = ConcurrencyNum if ConcurrencyNum < (executorCoreCnt * executorInstances) else (executorCoreCnt * executorInstances)

rddQueries = sc.parallelize(queryList, maxConcurrency)
rddQueriesWithId = rddQueries.zipWithUniqueId().map(lambda x: [x[1], (x[1], x[0])] )#, tokenstruct, runId]) # zipWithUniqueId is faster than ZipWithIndex but could create gaps in the Ids generated
rddQueriesWithId = rddQueriesWithId.partitionBy(maxConcurrency, lambda k: k ) # is this even needed?
print(rddQueriesWithId.glom().map(len).collect())  # Get length of each partition to check for even distribution of rows in the partitions. This will tell us if the number of queries are evenly distributed

# TODO adjust verbage of this
# print(f'Max currency of spark session is {(executorCoreCnt * executorInstances)}\nDefined concurrency is {concurrencyNum}\nCan only run {maxConcurrency} queries concurrently for this spark session')
displayHTML(f"""
<p><span style="font-size:20px;"><strong>Max currency of spark session is </strong><i><strong>{maxConcurrency}</strong></i></span></p>
<p><span style="font-size:20px;"><strong>Defined concurrency is </strong><i><strong>{ConcurrencyNum}</strong></i></span></p>
<p><span style="font-size:20px;"><strong>Can only run </strong><i><strong>{maxConcurrency}</strong></i><strong> queries concurrently for this spark session</strong></span></p>
""")

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 6, Finished, Available, Finished)

[12]


In [5]:
import requests

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces', headers=header)

while True:
    workspaceFound = False
    for workspace in response.json().get('value'):
        if workspace.get('displayName') == FabricDWWorkspaceName:
            fabricDWWorkspaceId = workspace.get('id')
            workspaceFound = True
            break
    
    if workspaceFound:
        break
    elif workspaceFound == False and response.json().get('continuationToken'):
        responseStatus = requests.request(method='get', url=response.json().get('continuationUri'), headers=header)
    else:
        print(f"Workspace was not found and no contination token found - {response.json()}")
        break

print(f'{fabricDWWorkspaceId = }\n{FabricDWWorkspaceName = }')

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 7, Finished, Available, Finished)

fabricDWWorkspaceId = '41a063a3-ef2a-4938-8570-3f5b2331f584'
FabricDWWorkspaceName = 'WS_DW_Query_Cost_Test_Take_Nine'


In [6]:
import requests

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}', headers=header)
workspaceName = response.json().get('displayName')

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}', headers=header)
capacityId = response.json().get('capacityId')

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/capacities', headers=header)
for capacity in response.json().get('value'):
    if capacity.get('id') == capacityId:
        capacityRegion = capacity.get('region')
        capacityName = capacity.get('displayName')
        capacitySku = capacity.get('sku')

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{fabricDWWorkspaceId}/warehouses', headers=header)
warehouse = [warehouse for warehouse in response.json().get('value') if warehouse.get('displayName') == FabricDWName][0]
fabricDWServer = warehouse.get('properties').get('connectionString')
warehouseId = warehouse.get('id')

print(f'{warehouseId = }\n{fabricDWServer = }\n{workspaceName = }\n{capacityId = }\n{capacityRegion = }\n{capacityName = }\n{capacitySku = }')

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 8, Finished, Available, Finished)

warehouseId = '179a51e7-8282-4172-9e2e-63aa5e0f8a5f'
fabricDWServer = 'n52dzjnhphme5jslxpytuo62ni-unr2aqjk544etblqh5nsgmpvqq.datawarehouse.fabric.microsoft.com'
workspaceName = 'WS_DW_Query_Cost_Test_Take_Nine'
capacityId = 'e36151a3-66ab-4bc5-b4f9-8a6e39f79d95'
capacityRegion = 'East US 2'
capacityName = 'fabricdemobam'
capacitySku = 'F32'


In [7]:
response = requests.request(method='get', url="https://prices.azure.com/api/retail/prices?$filter=skuName eq 'Fabric Capacity'", headers=header)
for capacity in response.json().get('Items'):
    if capacity.get('armRegionName') == capacityRegion.replace(' ', '').lower():
        costReserved = capacity.get('retailPrice') / 12 / 730 / 60 / 60 # get the amount per CU second
        costPayGo = costReserved / (156.334/262.80) # constant saving of ~41%. 156.334 is the resevered price of a region. 262.80 is the paygo price of a region
print(f'{costReserved = :.10f}\n{costPayGo = :.10f}') # per CU

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 9, Finished, Available, Finished)

costReserved = 0.0000297438
costPayGo = 0.0000499998


##### Define the queries to be executed. These are single line queries so use /* */ for commenting out code vs --

In [8]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={fabricDWServer};Database={FabricDWName}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

runId = str(uuid.uuid4())
runStartDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
runStartTimeEpoch = int(runStartDateTimeUTC.timestamp()*1000)

print(f'{runId = }\n{runStartDateTimeUTC = }\n{runStartTimeEpoch = }')

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 10, Finished, Available, Finished)

runId = '0d2d13ee-9aa1-4eaf-988f-528e12d628fa'
runStartDateTimeUTC = datetime.datetime(2024, 9, 24, 21, 44, 38, 830403, tzinfo=datetime.timezone.utc)
runStartTimeEpoch = 1727214278830


In [9]:
from delta.tables import *

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''SELECT @@VERSION AS DWVersion
                            ,@@SERVERNAME AS ServerGuid
                            ,DB_NAME() AS DWName
        ''')
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
        cursor.commit()
        resultSet = [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

        cursor.execute(f'''SELECT [is_vorder_enabled] AS IsVOrderEnabled, [data_lake_log_publishing_desc] AS DataLakeLogPublishingDesc
                            ,[data_lake_log_publishing] AS DataLakeLogPublishing, [create_date] AS DWCreateDate, [compatibility_level] AS CompatibilityLevel
                            FROM sys.databases 
                            WHERE [name] = '{FabricDWName}'
            ''')

        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
        cursor.commit()
        resultSet.extend([dict(zip(resultColumns, [str(col) for col in row])) for row in resultList])

        df = spark.createDataFrame([dict(zip(resultColumns, [str(col) for col in row])) for row in resultList])

dfRun = (df.withColumn('RunStartDateTimeUTC', F.lit(runStartDateTimeUTC).cast(TimestampType()))
            .withColumn('RunStartTimeEpoch', F.lit(runStartTimeEpoch).cast(LongType()))
            .withColumn('RunId', F.lit(runId).cast(StringType()))
            .withColumn('DWConnectionString', F.lit(fabricDWServer).cast(StringType()))
            .withColumn('QueriesExecutedCnt', F.lit(len(queryList)).cast(IntegerType()))
            .withColumn('RunConcurrency', F.lit(maxConcurrency).cast(IntegerType()))

            .withColumn('DWGuid', F.lit(warehouseId).cast(StringType())) # TODO
            .withColumn('WorkspaceName', F.lit(workspaceName).cast(StringType())) # TODO
            .withColumn('WorkspaceGuid', F.lit(fabricDWWorkspaceId).cast(StringType())) # TODO
            .withColumn('CapacityName', F.lit(capacityName).cast(StringType())) # TODO
            .withColumn('CapacityGuid', F.lit(capacityId).cast(StringType())) # TODO
            .withColumn('CapacitySKU', F.lit(capacitySku).cast(StringType())) # TODO
            .withColumn('CapacityRegion', F.lit(capacityRegion).cast(StringType())) # TODO

            .withColumn('RunEndDatetimeUTC', F.lit(None).cast(TimestampType()))
            .withColumn('RunEndTimeEpochMS', F.lit(None).cast(LongType()))
            .withColumn('RunDurationMS', F.lit(None).cast(LongType()))
            .withColumn('RunCUSeconds', F.lit(None).cast(DoubleType()))
            .withColumn('RunCostPayGo', F.lit(None).cast(DoubleType()))
            .withColumn('RunCostReserved', F.lit(None).cast(DoubleType()))
            
            .withColumn('CapacityDailyCUSeconds', F.lit(60*60*24*int(capacitySku.split('F')[1])))
            .withColumn('CapacityDailyCostPayGo', F.lit(costPayGo * 60*60*24*int(capacitySku.split('F')[1])))
            .withColumn('CapacityDailyCostReserved', F.lit(costReserved * 60*60*24*int(capacitySku.split('F')[1])))
        )


# # TODO add a merge so that if you need to rerun from this step, it doesn't insert a new record
dfRun.select("RunId"
        ,"DWConnectionString"
        ,"QueriesExecutedCnt"
        ,"RunConcurrency"
        ,"DWGuid"
        ,"WorkspaceName"
        ,"WorkspaceGuid"
        ,"CapacityName"
        ,"CapacityGuid"
        ,"CapacitySKU"
        ,"CapacityRegion"
        ,"CompatibilityLevel"
        ,"DWCreateDate"
        ,"DataLakeLogPublishing"
        ,"DataLakeLogPublishingDesc"
        ,"IsVOrderEnabled"
        ,"RunStartDateTimeUTC"
        ,"RunStartTimeEpoch"
        ,"RunEndDatetimeUTC"
        ,"RunEndTimeEpochMS"
        ,"RunDurationMS"
        ,"RunCUSeconds"
        ,"RunCostPayGo"
        ,"RunCostReserved"
        ,"CapacityDailyCUSeconds"
        ,"CapacityDailyCostPayGo"
        ,"CapacityDailyCostReserved"
    )

if spark.catalog.tableExists("RunResults"):

    dtRunResults = DeltaTable.forName(spark, "RunResults")

    (dtRunResults.alias('t')
        .merge(dfRun.alias('s')
            ,f't.runId = s.RunId'
            )
        .whenNotMatchedInsertAll()
    ).execute() 
else:
    dfRun.write.format('delta').mode('append').saveAsTable('RunResults')

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 11, Finished, Available, Finished)

In [10]:
from pyspark import SparkContext, SparkConf
import pyodbc 

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

def execute_query(iterator):
    queryMetrics = []
    for queryInfo in iterator:
        queryId = queryInfo[1][0]
        queryStatement = queryInfo[1][1]
        with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
            with conn.cursor() as cursor:
                queryStartDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
                startTime = int(round(time.time() * 1000))

                cursor.execute(queryStatement)
                endTime = int(round(time.time() * 1000))
                queryEndDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
                
                queryMessage = str(cursor.messages) if cursor.messages else ""
                resultSet = get_result_set(cursor)

                while cursor.nextset():
                    queryMessage += ",".join([str(cursor.messages) if cursor.messages else ""])
                    resultSet.append(get_result_set(cursor))
                
                statementId = ','.join(re.findall(r"Statement ID: \{([A-F0-9\-]+)\}", queryMessage)) if re.findall(r"Statement ID: \{([A-F0-9\-]+)\}", queryMessage) else ""
                queryHash = ','.join(re.findall(r"Query hash: (0x[A-F0-9]+)", queryMessage)) if re.findall(r"Query hash: (0x[A-F0-9]+)", queryMessage) else ""
                distributionRequestId = ','.join(re.findall(r"Distributed request ID: \{([A-F0-9\-]+)\}", queryMessage)) if re.findall(r"Distributed request ID: \{([A-F0-9\-]+)\}", queryMessage) else ""
                resultSetJsonString = json.dumps(resultSet)

                cursor.commit()

                queryId = str(uuid.uuid4())
                queryMetrics.append([runId, queryId, queryStatement, queryStartDateTimeUTC, queryEndDateTimeUTC
                        ,queryMessage, startTime, endTime, endTime - startTime
                        ,statementId, queryHash, distributionRequestId, resultSetJsonString
                        ])
    return queryMetrics

queriesExecuted = rddQueriesWithId.mapPartitions(execute_query)

dfQueriesExecuted = queriesExecuted.toDF(schema=StructType([
    StructField("RunId", StringType(), False),
    StructField("QueryId", StringType(), False),
    StructField("QueryStatement", StringType(), False),
    StructField("QueryStartDateTimeUTC", TimestampType(), False),
    StructField("QueryEndDateTimeUTC", TimestampType(), False),
    StructField("ReturnMessage", StringType(), False),
    StructField("QueryStartTimeEpochMS", LongType(), False),
    StructField("QueryEndTimeEpochMS", LongType(), False),
    StructField("QueryDurationMS", LongType(), False),
    StructField("StatementId", StringType(), False),
    StructField("QueryHash", StringType(), False),
    StructField("DistributionRequestId", StringType(), False),
    StructField("ResultSet", StringType(), False)
    ]))

dfFinal = dfQueriesExecuted.withColumn('QueryCUSeconds', F.lit(None).cast(DoubleType())).withColumn('QueryCostPayGo', F.lit(None).cast(DoubleType())).withColumn('QueryCostReserved', F.lit(None).cast(DoubleType()))
dfFinal.write.format('delta').mode('append').saveAsTable('QueryResults')

runEndDateTimeUTC = datetime.datetime.now(datetime.timezone.utc)
runEndTimeEpoch = int(runEndDateTimeUTC.timestamp()*1000)

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 12, Finished, Available, Finished)

In [11]:
statementList = spark.sql(f'SELECT ARRAY_JOIN(COLLECT_SET(CONCAT("\\"", StatementId, "\\"")), ", ") AS Statements FROM (SELECT EXPLODE(SPLIT(StatementId, ",")) AS StatementId FROM QueryResults WHERE runId = "{runId}") AS a ').collect()[0].asDict().get('Statements')
# We have to explode by statement ids since a sql query may have multiple queries within it
queriesExecutedCnt = spark.sql(f'SELECT COUNT(StatementId) AS QueryCnt FROM (SELECT EXPLODE(SPLIT(StatementId, ",")) AS StatementId FROM QueryResults WHERE runId = "{runId}") AS a ').collect()[0].asDict().get('QueryCnt') 
print(f'{runId = }\n{statementList = }\n{queriesExecutedCnt = }')

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 13, Finished, Available, Finished)

runId = '0d2d13ee-9aa1-4eaf-988f-528e12d628fa'
statementList = '"33A655DC-EA1A-4130-A272-78FC2A918E53", "49D14862-51AB-4173-BE2A-A7821D4E07BF", "CE110DE9-E753-43C0-80C8-73D7E5B150ED", "4F26DC75-A7D7-435D-B70D-7F03AD0214D6", "69516C44-4A17-439F-B4EC-3F6619155748", "29634577-8C4B-4208-A223-DE494F7A5CD1", "7ECD8E14-793D-436A-814E-5521D4F9436A", "A73C7CFF-171D-4581-A02A-06C5430B192F", "38E656B1-65BD-4DAD-8421-A23938EC9204", "0A456AE7-BBB6-4199-BEEE-4AA50DB4C2B3", "54DB14CB-19F4-48AC-B3C4-86E580E4A238", "5DA623E1-EDDB-4CDA-8460-8AF7557BCE68", "7A5E64B9-4907-4236-8502-AC9E6B47D0A2", "6A4938BD-F2EF-4D0E-8A9C-115FD623CE00", "A63595D5-0C2F-44C4-A411-F363258D5AD6"'
queriesExecutedCnt = 15


In [12]:
from delta.tables import *

dtRunResults = DeltaTable.forName(spark, "RunResults")

df_final = spark.createDataFrame(data=[(runEndDateTimeUTC, runEndTimeEpoch, )], schema=['RunEndDateTimeUTC', 'RunEndTimeEpochMS'])

(dtRunResults.alias('t')
    .merge(df_final.alias('s')
        ,f't.RunId = "{runId}"'
        )
    .whenMatchedUpdate(set=
        {'RunEndDatetimeUTC': 's.RunEndDateTimeUTC'
        ,'RunEndTimeEpochMS': 's.RunEndTimeEpochMS'
        ,'RunDurationMS': 's.RunEndTimeEpochMS - t.RunStartTimeEpoch'
        }
        )
).execute() 

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 14, Finished, Available, Finished)

In [13]:
import sempy.fabric as fabric
from datetime import timedelta
from pyspark.sql.functions import col, lit, sum, min, max
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType

def get_capacity_metrics_usage(time_point, operation_id_list):
	
	schema = StructType([
		StructField("Items[ItemId]", 	StringType(), 		True),
		StructField("Items[ItemKind]", 	StringType(), 		True),
		StructField("Items[ItemName]", 	StringType(), 		True),
		StructField("TimePointBackgroundDetail[OperationStartTime]", 	TimestampType(), 	True),
		StructField("TimePointBackgroundDetail[OperationEndTime]", 		TimestampType(), 	True),
		StructField("TimePointBackgroundDetail[OperationId]", 			StringType(), 		True),
		StructField("Sum_CUs", 		DoubleType(), 		True),
		StructField("Sum_Duration", IntegerType(), 		True)
	])

	dax_command = f"""
	DEFINE
		MPARAMETER 'CapacityID' 	= "{capacityId}"
		MPARAMETER 'TimePoint' 		= (DATE({time_point.year}, {time_point.month}, {time_point.day}) + TIME({time_point.hour}, {time_point.minute}, {time_point.second}))

		VAR __Var_CapacityId	= {{"{capacityId}"}}
		VAR __Var_OperationId	= {{{statementList}}}

		VAR __Filter_OperationId 	= TREATAS(__Var_OperationId, 'TimePointBackgroundDetail'[OperationId])
		VAR __Filter_CapacityId 	= TREATAS(__Var_CapacityId, 'Capacities'[capacityId])

		VAR OperationCUs = 
			SUMMARIZECOLUMNS(
				'Items'[ItemId],
				'Items'[ItemKind],
				'Items'[ItemName],
				'TimePointBackgroundDetail'[OperationStartTime],
				'TimePointBackgroundDetail'[OperationEndTime],
				'TimePointBackgroundDetail'[OperationId],
				__Filter_OperationId,
				__Filter_CapacityId,
				"Sum_CUs", CALCULATE(SUM('TimePointBackgroundDetail'[Total CU (s)])),
				"Sum_Duration", CALCULATE(SUM('TimePointBackgroundDetail'[Duration (s)]))
			)

	EVALUATE
		OperationCUs
	"""

	df_dax = fabric.evaluate_dax(dax_string = dax_command, dataset = CapacityMetricsDataset, workspace = CapacityMetricsWorkspace)

	df = (spark.createDataFrame(data=df_dax, schema=schema).withColumn("TimePoint", lit(time_point)).select(
			col("TimePoint")
			,col("Items[ItemId]").alias("ItemId")
			,col("Items[ItemKind]").alias("ItemKind")
			,col("Items[ItemName]").alias("ItemName")
			,col("TimePointBackgroundDetail[OperationStartTime]").alias("StartTime")
			,col("TimePointBackgroundDetail[OperationEndTime]").alias("EndTime")
			,col("TimePointBackgroundDetail[OperationId]").alias("OperationId")
			,col("Sum_CUs").cast(DoubleType()).alias("Sum_CUs")
			,col("Sum_Duration").cast(IntegerType()).alias("Sum_Duration"))
		)

	return df

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 15, Finished, Available, Finished)

In [14]:
'''
Get the capacity usage for the queries that were executed.
This ensures that the background operations are captured no matter what time they are run as they are smoothed over a 24 hour time period.
Next, filter that down to the distinct records. This is necessary because a record may show up in the today and tomorrow datasets depending on the time it was run.
Nest, aggregate the records into a single record. This is necessary becuase some operations will have two entries, one under the executing user and one under the user "System".
'''

# # Continues to query the metrica app to get the data. Data can delayed by a few minutes.
# # We retry every minute until 15 minutes has passed.
for retryCnt in range(15):
    df_today    = get_capacity_metrics_usage(runStartDateTimeUTC, statementList)
    df_tomorrow = get_capacity_metrics_usage(runStartDateTimeUTC + timedelta(hours = 24), statementList)
    df_all_days = df_today.unionAll(df_tomorrow)
    df_distinct = df_all_days.select('ItemId', 'ItemKind', 'ItemName', 'StartTime', 'EndTime', 'OperationId', 'Sum_CUs', 'Sum_Duration').distinct()
    df_final    = df_distinct.groupBy('ItemId', 'ItemKind', 'ItemName', 'OperationId').agg(min("StartTime").alias("StartTime"), max("EndTime").alias("EndTime"), sum("Sum_CUs").alias("QueryCUSeconds"), sum("Sum_Duration").alias("SumDuration"))

    print(f'{df_final.count()} queries of the {queriesExecutedCnt} that have been found in the capacity metrics model. ', end='')
    if df_final.count() == queriesExecutedCnt:
        break
    else:
        print('Sleeping for a minute...')
        time.sleep(60)

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 16, Finished, Available, Finished)

0 queries of the 15 that have been found in the capacity metrics model. Sleeping for a minute...
0 queries of the 15 that have been found in the capacity metrics model. Sleeping for a minute...
0 queries of the 15 that have been found in the capacity metrics model. Sleeping for a minute...
0 queries of the 15 that have been found in the capacity metrics model. Sleeping for a minute...
15 queries of the 15 that have been found in the capacity metrics model. 

In [15]:
from pyspark.sql.functions import sum

dfQueryResults = spark.table("QueryResults")

dfQueryResultsCleansed = (dfQueryResults.join(df_final, dfQueryResults.StatementId.contains(df_final.OperationId)) 
    .filter(dfQueryResults.RunId == runId) 
    .groupBy(dfQueryResults.RunId, dfQueryResults.StatementId) 
    .agg(sum(df_final.QueryCUSeconds).alias("QueryCUSeconds"), sum(df_final.SumDuration).alias("SumDuration")
    ))

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 17, Finished, Available, Finished)

##### Update the QueryResults table with the CUSeconds and QueryCost derived from the Capacity Metrics App

In [16]:
from delta.tables import *

dtQueryResults = DeltaTable.forName(spark, "QueryResults")

(dtQueryResults.alias('t')
    .merge(dfQueryResultsCleansed.alias('s')
        ,f't.runId = s.RunId AND t.StatementId = s.StatementId'
        )
    .whenMatchedUpdate(set=
        {'QueryCUSeconds': 's.QueryCUSeconds'
        ,'QueryCostPayGo': f's.QueryCUSeconds * {costPayGo}'
        ,'QueryCostReserved': f's.QueryCUSeconds * {costReserved}'
        }
        )
).execute() 

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 18, Finished, Available, Finished)

##### Update the RunResults table with cost of run

In [17]:
from delta.tables import *

dtRunResults = DeltaTable.forName(spark, "RunResults")
dtRunResultsCleansed = spark.sql(f'SELECT SUM(COALESCE(QueryCUSeconds, 0)) AS RunCUSeconds, SUM(COALESCE(QueryCostPayGo, 0)) AS RunCostPayGo, SUM(COALESCE(QueryCostReserved, 0)) AS RunCostReserved FROM QueryResults WHERE RunId = "{runId}"')

(dtRunResults.alias('t')
    .merge(dtRunResultsCleansed.alias('s')
        ,f't.RunId = "{runId}"'
        )
    .whenMatchedUpdate(set=
        {'RunCUSeconds': 's.RunCUSeconds'
        ,'RunCostPayGo': f's.RunCUSeconds * {costPayGo}'  # This could be different looking at the tables separately due to rounding
        ,'RunCostReserved': f's.RunCUSeconds * {costReserved}'  # This could be different looking at the tables separately due to rounding
        }
        )
).execute()

StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 19, Finished, Available, Finished)

In [18]:
displayHTML(f"""<script src="https://cdn.rawgit.com/google/code-prettify/master/loader/run_prettify.js"></script>
<p style="margin-bottom:0"><span style="font-size:20px;"><strong>/*<br>Reference T-SQL - </strong></span><span style="font-size:20px;"><strong>See running sql statements on the DW. Used to verify query(s) are executing and that the concurrency is working correctly.<br>*/</strong></span></p>
<pre class="prettyprint"><p style="margin-top:0;">SELECT	d.name AS 'database_name'
	,s.login_name
	,r.[session_id]
	,r.start_time
	,r.STATUS
	,r.total_elapsed_time
	,r.command
	,CASE /* Uses statement start and end offset to figure out what statement is running */
		WHEN r.[statement_start_offset] > 0
			THEN
				/* The start of the active command is not at the beginning of the full command text */
				CASE r.[statement_end_offset]
					WHEN - 1
						THEN
							/* The end of the full command is also the end of the active statement */
							SUBSTRING(t.TEXT, (r.[statement_start_offset] / 2) + 1, 2147483647)
					ELSE
						/* The end of the active statement is not at the end of the full command */
						SUBSTRING(t.TEXT, (r.[statement_start_offset] / 2) + 1, (r.[statement_end_offset] - r.[statement_start_offset]) / 2)
					END
		ELSE
			/* 1st part of full command is running */
			CASE r.[statement_end_offset]
				WHEN - 1
					THEN
						/* The end of the full command is also the end of the active statement */
						RTRIM(LTRIM(t.[text]))
				ELSE
					/* The end of the active statement is not at the end of the full command */
					LEFT(t.TEXT, (r.[statement_end_offset] / 2) + 1)
				END
		END AS [executing_statement]
	,t.[text] AS [parent_batch]
	,s.[program_name]
	,r.query_hash
	,r.query_plan_hash
	,r.dist_statement_id
	,r.[label]
	,s.client_interface_name
	,r.[sql_handle]
	,c.client_net_address
	,c.connection_id
FROM	sys.dm_exec_requests r
CROSS APPLY sys.[dm_exec_sql_text](r.[sql_handle]) t
JOIN	sys.dm_exec_sessions s ON r.session_id = s.session_id
JOIN	sys.dm_exec_connections c ON s.session_id = c.session_id
JOIN	sys.databases d ON d.database_id = r.database_id
WHERE	r.dist_statement_id != '00000000-0000-0000-0000-000000000000'
AND	r.session_id <> @@SPID
AND	s.program_name NOT IN ('QueryInsights','DMS')
</pre></p>
""")


StatementMeta(, 535874ce-8165-4ce3-bb88-950999d34045, 20, Finished, Available, Finished)